# Workspaces - View

**Author**: Melissa Coates  

**Last updated and tested**: August 22, 2022 

**Purpose:** This notebook contains a few ad hoc scripts to view Power BI workspace information:

- View a list of workspaces - user scope
- View a list of workspaces - organization scope
- View workspace info with artifiacts - organization scope

**Type of authentication**: You can log in with either:

- A domain user account (requires Power BI administrator permissions if using the organization scope), or
- A service principal (with permission to run admin APIs)

**How data is being accessed:** The notebook uses:

- Power BI Management Module for authentication: Connect-PowerBIServiceAccount
- Power BI Management Module for extracting the workspace information: Get-PowerBIWorkspace

**Type of script**: Interactive, ad hoc (non-scheduled). This notebook is a guided learning experience to get familiar with how to query the workspace metadata. It's not production-ready.

This notebook is not designed to do a 'Run all.' Rather, it's designed to be run cell-by-cell, based on the scenario. More information, including prerequisites, is included at the bottom of this notebook.

## **Log in to the Power BI Service**
---
**Update before executing:**

- **Type of authentication:** 
    - **Line 2 - Choice for type of auth:** Input either '**User**' or '**SP**' without the quotes. This tells the script whether or not to log in as a user or a service principal.
- **If using user authentication:**
    - **Line 3 - Email address:** Input the email address for the user. This user must be a Power BI administrator. MFA can't be enabled for this account.
- **If using service principal authentication:**
    - **Line 4 - Azure AD app ID**: Input the app ID (aka client ID). 
    - **Line 5 - Azure tenant ID**: Input the tenant ID (aka directory ID).

**Interactive prompts:**

The script will interactively prompt you for the sensitive information (because hard-coding of passwords and secrets isn't secure):

- **If using user authentication:**
    - It will ask for the user's password.
- **If using service principal authentication:**
    - It will ask for the app secret.

**Comments:**

- This script uses the PowerShell cmdlet: [<u>Connect-PowerBIServiceAccount</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.profile/connect-powerbiserviceaccount).
- After signing in, the access token is active for one hour.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$TypeOfAuth = 'User'  #Options: User or SP
[string]$DomainUserEmailAddr = 'user@domain.com'
[string]$AzureADAppID = '1234-1234'
[string]$AzureTenantID = '1234-1234'
#--------------------------------------------

if ($TypeOfAuth -eq 'User')
{
    Write-Verbose "Domain user authentication will be used." -Verbose
    [securestring]$DomainUserPW = Read-Host -Prompt "Input password for $DomainUserEmailAddr" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($DomainUserEmailAddr, $DomainUserPW)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj
}
elseif ($TypeOfAuth -eq 'SP')
{
    Write-Verbose "Service principal authentication will be used." -Verbose
    [securestring]$AzureADAppSecret = Read-Host -Prompt "Input secret for $AzureADAppID" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($AzureADAppID, $AzureADAppSecret)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj -ServicePrincipal -Tenant $AzureTenantID
}
else 
{
    Write-Verbose "There's a problem. The Type of Auth variable input was $TypeOfAuth. Expected values are User or SP." -Verbose
}

## **View Workspace Info: User Scope**  
---
## <span style="font-size: 14px;">The next script will allow you to view a list of workspaces, based on permissions for the signed-in user or service principal. This is useful when you're looking for a list of workspaces you have access to.</span>  

**Comments:**

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIWorkspace</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.workspaces/get-powerbiworkspace).
- The first few results are shown, to avoid returning too much data.

In [ ]:
Get-PowerBIWorkspace  -First 5

## **View Workspace Info: Organization Scope**

* * *

## <span style="font-size: 14px;">The next scripts will allow you to view a list of workspaces in the entire tenant.</span>

**Comments:**

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIWorkspace</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.workspaces/get-powerbiworkspace)
- **Permissions required**: Power BI administrator permissions, or a service principal with permission to use read-only admin APIs, are required for the organization scope
- Note that when run with the organization scope, more attributes are available, including the user workspace access rights.
- The first few results are shown, to avoid returning too much data. (See the Workspaces-Export notebook instead, which is easier to use for large tenant with a lot of workspaces.)

In [ ]:
Get-PowerBIWorkspace -Scope Organization -First 2

### **Active workspaces**

The next script will return active workspaces. It also reformats the results and displays only 3 columns.

The first few results are shown, to avoid returning too much data. (See the Workspaces-Export notebook instead, which is easier to use for large tenant with a lot of workspaces.)

In [ ]:
Get-PowerBIWorkspace -Scope Organization -First 3 | Where-Object State -eq 'Active' | Format-Table Name, Type, IsOnDedicatedCapacity

### **Orphaned workspaces**

The next script will find orphaned workspaces (a workspace without a workspace administrator assigned).

In [ ]:
Get-PowerBIWorkspace -Scope Organization -Orphaned

### **Deleted workspaces**

The next script will find workspaces that have been deleted by a workspace administrator, but haven't yet been fully removed by the Power BI service. If the deletion was done in error, they can be restored (using the Restore-PowerBIWorkspace cmdlet).

In [ ]:
Get-PowerBIWorkspace -Scope Organization -Deleted

### **Number of workspaces in the tenant**

In [ ]:
$workspaces = Get-PowerBIWorkspace -Scope Organization 
Write-Verbose "The total number of Power BI workspaces in the tenant: $($workspaces.count)" -Verbose 

In [ ]:
$workspaces = Get-PowerBIWorkspace -Scope Organization | Where-Object State -eq 'Active' 
Write-Verbose "The number of active Power BI workspaces in the tenant: $($workspaces.count)" -Verbose 

## **View Workspace Info with Artifacts: Organization Scope**  
---
## <span style="font-size: 14px;">The next scripts will allow you to view a list of workspaces in the entire tenant, as well as a list of the artifacts within the workspace.</span>  

**Comments:**

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIWorkspace</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.workspaces/get-powerbiworkspace)
- **Permissions required**: Power BI administrator permissions, or a service principal with permission to use read-only admin APIs, are required for the organization scope
- Note that when run with the organization scope, more attributes are available, including the user workspace access rights.

### **List of reports**

First, check how report results are returned (using the [<u>Get-PowerBIReport</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.reports/get-powerbireport) cmdlet). Note that the results don't explicitly state which workspace the report is in.

The first few results are shown, to avoid returning too much data. (See the Workspaces-Export notebook instead, which is easier to use for large tenant with a lot of content.)

In [ ]:
Get-PowerBIReport -Scope Organization -First 2

### **List of reports for one workspace**

Instead, use the Get-PowerBIWorkspace cmdlet. With the -Include parameter, specify that we want to see a list of reports. This way, we have more clear information.

**Update before executing:**

- **Line 2 - workspace name** (used so the list of results is short)

**Comments**:

- The first information returned is the workspace info. After that, a JSON array \[ \] of reports is listed.
- The dashboards, datasets, dataflows, and workbooks arrays are null because we didn't ask for that data.

In [ ]:
#-----------------INPUT AREA-----------------
$WorkspaceName = 'Sales Analytics'
#--------------------------------------------

Get-PowerBIWorkspace -Scope Organization -Name $WorkspaceName -Include Reports | ConvertTo-JSON -Depth 10

### **List of reports for one workspace**

With the -Include parameter, specify that we want to see all artifacts. 

**Update before executing:**

- **Line 2 - workspace name** (used so the list of results is short)

**Comments**:

- The first information returned is the workspace info. After that, a JSON array \[ \] of reports is listed, followed by dashboards, datasets, dataflows, and workbooks.

In [ ]:
#-----------------INPUT AREA-----------------
$WorkspaceName = 'Sales Analytics'
#--------------------------------------------

Get-PowerBIWorkspace -Scope Organization -Name $WorkspaceName -Include All | ConvertTo-JSON -Depth 10

## **Observations**
---
The above technique is good for obtaining information about what's in your tenant. However, in a large tenant with a lot of data, it's much more efficient to use the [<u>Scanner APIs</u>](https://docs.microsoft.com/en-us/power-bi/enterprise/service-admin-metadata-scanning) instead.

## **More Information About This File**

* * *

## <span style="font-size:14px;">Prerequisites for running this script:</span>

- The [Power BI Management module](https://www.powershellgallery.com/packages/MicrosoftPowerBIMgmt) for PowerShell needs to be installed.
    
- The [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension needs to be installed and enabled in VS Code and/or Azure Data Studio.
    
- Either a user account with Power BI administrator permissions, or a service principal.
    

This script **interactively prompts** for the sensitive information to be passed into a credential object. Therefore, it's only useful for ad hoc purposes. A different technique is used for securely storing the credentials when a script is to be operationalized and scheduled.

The original / most recent version of this notebook can be found on Melissa Coates' [Power-BI-Admin repository in GitHub](https://github.com/sqlchick/Power-BI-Admin).